# 03. 음성 모델 학습
### Mell + LSTM 다중 분류 (4개 클래스)
- 클래스 0: 비강조
- 클래스 1: 침묵+강조
- 클래스 2: 음 높낮이 변화+강조
- 클래스 3: 크게 말함+강조


In [ ]:
import os
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

# ==========================================
# ▼ [설정] 저장할 파일명 정의
# ==========================================
DATA_FOLDER = r"/home/stu/ai_project/voice_data"
SAVE_MODEL_PATH = "best_bi_lstm.pth"  


SR = 16000
N_FFT = 1024
HOP_LENGTH = 512
WINDOW_SIZE = 50
BATCH_SIZE = 256 
EPOCHS = 100     
LR = 0.001
NUM_CLASSES = 4

# ==========================================
# 1. 모델 정의 (Winner: Bi-LSTM)
# ==========================================
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_classes=4):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# ==========================================
# 2. 데이터 로드 (Mel-Spectrogram)
# ==========================================
def create_dataset():
    print("📂 데이터 로딩 및 전처리 중... (Mel-Spectrogram)")
    all_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.wav')]
    train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)

    def process(files):
        X_list, y_list = [], []
        for f in files:
            try:
                path = os.path.join(DATA_FOLDER, f)
                y, _ = librosa.load(path, sr=SR)
                y = librosa.util.normalize(y)
                
                # Mel-Spec
                mel = librosa.feature.melspectrogram(y=y, sr=SR, n_mels=80, n_fft=N_FFT, hop_length=HOP_LENGTH)
                feat = librosa.power_to_db(mel, ref=np.max).T
                
                scaler = StandardScaler()
                feat_norm = scaler.fit_transform(feat)
                
                # Auto Labeling
                num_frames = len(feat_norm)
                labels = np.zeros(num_frames, dtype=int)
                rms = librosa.feature.rms(y=y, frame_length=N_FFT, hop_length=HOP_LENGTH)[0][:num_frames]
                cent = librosa.feature.spectral_centroid(y=y, sr=SR, n_fft=N_FFT, hop_length=HOP_LENGTH)[0][:num_frames]
                
                feat_norm = feat_norm[:len(rms)]; labels = labels[:len(rms)]
                rms_z = (rms - np.mean(rms)) / (np.std(rms) + 1e-6)
                cent_z = (cent - np.mean(cent)) / (np.std(cent) + 1e-6)
                
                for i in range(len(labels)):
                    if i > 5 and np.mean(rms_z[i-5:i]) < -0.5 and rms_z[i] > 0: labels[i] = 1
                    elif rms_z[i] > 1.5: labels[i] = 3
                    elif cent_z[i] > 1.5: labels[i] = 2
                
                # Windowing
                for i in range(0, len(feat_norm) - WINDOW_SIZE, 10):
                    X_list.append(feat_norm[i : i + WINDOW_SIZE])
                    y_list.append(labels[i + WINDOW_SIZE - 1])
            except: pass
        return np.array(X_list), np.array(y_list)

    X_train, y_train = process(train_files)
    X_val, y_val = process(val_files)
    
    return X_train, y_train, X_val, y_val

# ==========================================
# 3. 학습 및 저장 실행
# ==========================================
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    X_train, y_train, X_val, y_val = create_dataset()
    
    cls = np.unique(y_train)
    w = compute_class_weight('balanced', classes=cls, y=y_train)
    cw = torch.ones(NUM_CLASSES).to(device)
    for c, weight in zip(cls, w): cw[int(c)] = weight
    
    train_dl = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train)), batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val)), batch_size=BATCH_SIZE, shuffle=False)
    
    input_dim = X_train.shape[2]
    model = BiLSTM(input_dim, num_classes=NUM_CLASSES).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss(weight=cw)
    
    best_f1 = 0.0
    
    print(f"\n🚀 최종 모델 학습 시작 ({EPOCHS} Epochs)...")
    
    for ep in range(EPOCHS):
        model.train()
        for bx, by in train_dl:
            bx, by = bx.to(device), by.to(device)
            optimizer.zero_grad()
            loss = criterion(model(bx), by)
            loss.backward()
            optimizer.step()
            
        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for bx, by in val_dl:
                bx = bx.to(device)
                outputs = model(bx)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                targets.extend(by.numpy())
        
        score = f1_score(targets, preds, average='macro', zero_division=0)
        
        if score > best_f1:
            best_f1 = score
            torch.save(model.state_dict(), SAVE_MODEL_PATH) 
            print(f"  [Ep {ep+1}] New Best F1: {score:.4f} -> Saved!")
            
    print("\n" + "="*50)
    print(f"🎉 학습 완료! 저장된 파일: {SAVE_MODEL_PATH}")
    print(f"🏆 최종 최고 F1 Score: {best_f1:.4f}")
    print("="*50)
    print("👉 이제 추론(Inference) 코드에서 MODEL_PATH = 'best_bi_lstm.pth' 로 바꿔서 쓰세요!")

📂 데이터 로딩 및 전처리 중... (Mel-Spectrogram)

🚀 최종 모델 학습 시작 (100 Epochs)...
  [Ep 1] New Best F1: 0.6276 -> Saved!
  [Ep 2] New Best F1: 0.6343 -> Saved!
  [Ep 4] New Best F1: 0.6971 -> Saved!
  [Ep 12] New Best F1: 0.7036 -> Saved!
  [Ep 13] New Best F1: 0.7109 -> Saved!
  [Ep 18] New Best F1: 0.7326 -> Saved!
  [Ep 29] New Best F1: 0.7343 -> Saved!
  [Ep 37] New Best F1: 0.7358 -> Saved!
  [Ep 52] New Best F1: 0.7422 -> Saved!

🎉 학습 완료! 저장된 파일: best_bi_lstm.pth
🏆 최종 최고 F1 Score: 0.7422
👉 이제 추론(Inference) 코드에서 MODEL_PATH = 'best_bi_lstm.pth' 로 바꿔서 쓰세요!
